In [259]:
import pandas as pd
import numpy as np
import random
from random import randrange

In [260]:
parent = '../datasets/'
dataset_name = 'abalone'
location = parent + dataset_name + '/' + dataset_name + '.csv'

data = pd.read_csv(location, header=0)
print(len(data))
unique_ages = [1, 2, 25, 26, 29]
data = data[~data['age'].isin(unique_ages)].reset_index(drop=True)
print(len(data))

data.head()

4177
4172


,sex,length,diameter,height,whole weight,shucked weight,viscera weight,shell weight,age
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [261]:
# 1, 2, 25, 26, 29
# 24, 27

In [262]:
## --- ABALONE DATASET --- ###

double_ages = []
double_rows = []
for age in set(data['age']):        
    if len(data[data['age'] == age]) == 2:
        double_ages.append(age)
        df_aux = data[data['age'] == age].reset_index(drop=True)
        double_rows.append(df_aux.iloc[[0]])
        double_rows.append(df_aux.iloc[[1]])
        
print(len(data))
data = data[~data['age'].isin(double_ages)].reset_index(drop=True)
print(len(data))

4172
4168


In [263]:
print(len(set(data['age'])))
print(set(data['age']))

21
{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}


In [264]:
### --- NURSERY DATASET --- ###

# df_aux = data[data['nursery'] == 'recommend'].reset_index(drop=True)
# row_test = df_aux.iloc[[0]]
# row_train = df_aux.iloc[[1]]

# print(len(data))
# data = data[data['nursery'] != 'recommend'].reset_index(drop=True)
# print(len(data))

In [265]:
perc = 1
test_param = 1 - perc*0.01
print(test_param)

0.99


In [266]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=test_param, random_state=42)
for train_index, test_index in split.split(data, data[data.columns[-1]]):
    train_set = data.loc[train_index]
    test_set = data.loc[test_index]

train_set = train_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

In [272]:
print('Train:', len(train_set))
print('Test', len(test_set))
len_total = len(train_set) + len(test_set)
print('Total:', len_total)
print('%train:', len(train_set)/len_total)

Train: 50
Test 4122
Total: 4172
%train: 0.011984659635666348


In [268]:
i = 0
while i < len(double_rows):
    idx = randrange(len(test_set))
    train_set = pd.concat([train_set.iloc[:idx], double_rows[i], train_set.iloc[idx:]]).reset_index(drop=True)
    i += 1
    idx = randrange(len(test_set))
    test_set = pd.concat([test_set.iloc[:idx], double_rows[i], test_set.iloc[idx:]]).reset_index(drop=True)
    i += 1

In [269]:
diff = set(test_set['age']) - set(train_set['age'])
diff

{3, 18, 19, 20, 21, 22, 23}

In [270]:
indexes = []
for age in diff:    
    # get random row in test set containing age missing in train set 
    aux = test_set[test_set['age'] == age]
    idx = random.choice(list(aux.index))
    indexes.append(idx)
    row = test_set.iloc[[idx]]
        
    # append to train set in random position
    new_idx = randrange(len(train_set))
    train_set = pd.concat([train_set.iloc[:new_idx], row, train_set.iloc[new_idx:]]).reset_index(drop=True)
    
for idx in indexes:
    # drop from test set
    test_set = test_set.drop(index=idx)

In [271]:
print(len(set(train_set['age'])))
print(len(set(test_set['age'])))

23
23


In [273]:
D = list(map(list, train_set.itertuples(index=False)))

train_location = '../datasets/' + dataset_name + '/extreme/' + str(perc) + '/' + dataset_name + '_train_perc' + str(perc) + '.csv'
with open(train_location, "w") as f:
    f.write(','.join(data.columns.tolist()) + '\n')
    for value in D:
        a = ','.join([str(i if type(i) == str else round(i,2)).strip() for i in value[0:]]) + '\n'
        f.write(a)

In [274]:
D = list(map(list, test_set.itertuples(index=False)))

test_location = '../datasets/' + dataset_name + '/extreme/' + str(perc) + '/' + dataset_name + '_test_perc' + str(perc) + '.csv'
with open(test_location, "w") as f:
    f.write(','.join(data.columns.tolist()) + '\n')
    for value in D:
        a = ','.join([str(i if type(i) == str else round(i,2)).strip() for i in value[0:]]) + '\n'
        f.write(a)